In [ ]:
%matplotlib inline 
import numpy as np
from matplotlib import pyplot as plt
#Generate Data
#
N = 784
dim = 784
lamda = 0.00001
mu=lamda
np.random.seed(50)
w = np.matrix(np.random.multivariate_normal([0.0]*dim, np.eye(dim))).T
X = np.matrix(np.random.multivariate_normal([0.0]*dim, np.eye(dim), size = N))
y = X*w
L = (np.linalg.svd(X)[1][0])**2
max_iter = 10000
b=np.zeros(N)
for i in range(len(b)):
  b[i]=y[i][0]
#Tim gia tri
n=dim
B=np.array(X)
def giatri(x):
  gt=0
  for i in range(0,n):
    gt+=0.5*((np.matmul(B,x)-b)[i])**2+mu*abs(x[i])
  return gt

In [ ]:
## Define the objective, subgradient oracle, proximal operator, and gradient for smoothed function (Huber)  
def obj(w):
    r = X*w-y;
    return 0.5*np.sum(np.multiply(r,r)) +  lamda * np.sum(np.abs(w))
def subgrad(w):
    return  X.T*(X*w-y) + lamda*np.sign(w) 

def f_grad(w):
    return  X.T*(X*w-y) 

def soft_threshod(w,mu):
    return np.multiply(np.sign(w), np.maximum(np.abs(w)- mu,0))  

def smooth_grad(w, mu):
    temp = np.multiply((np.abs(w)<=mu),w/mu) + np.multiply((np.abs(w)>mu),np.sign(w)) 
    return X.T*(X*w-y) + lamda * temp

In [ ]:
## Nesterovs' Accelerated Proximal Gradient with Backtracking
import time
w = np.matrix([1.0]*dim).T
v = w
obj_APG_LS = []
L=1
gamma = 1/L
beta = 1.2
start_time = time.time()
time_APG=[]
value_APG=[]
t=0
while (float(time.time() - start_time)<2200):
    obj_val = obj(w)
    w_prev = w
    delta = 1
    while (delta>1e-3):
        gamma = 1/L
        w = v - gamma * f_grad(v)    
        w = soft_threshod(w,lamda * gamma)
        delta = obj(w) - obj_val - f_grad(w_prev).T*(w-w_prev)- (L/2) * np.linalg.norm(w-w_prev)**2
        L = L*beta
    L = L/beta    
    v = w + t/(t+3) * (w - w_prev)
    obj_APG_LS.append(obj_val.item())
    if (t%500==0):
        print('iter= {},\tobjective= {:3f}'.format(t, obj_val.item()))
        time_APG.append(float(time.time() - start_time))
        value_APG.append(obj_val.item())
    t+=1

In [ ]:
## ADMM
w = np.matrix([1.0]*dim).T
z = w
u = w
obj_ADMM = []
rho = 5
import time
times_ADMM=[]
start_time = time.time()
value_ADMM=[]
t=0
#for t in range(0, max_iter):
while float(time.time() - start_time)<10:
    obj_val = obj(w)
    w = np.linalg.inv((X.T)*X + rho*np.identity(dim))*(X.T*y + rho*z - u )
    z= soft_threshod(w + u/rho,lamda/rho)
    u = u + rho * (w-z)
    obj_ADMM.append(obj_val.item())
    if (t%500==0):
        print('iter= {},\tobjective= {:3f}'.format(t, obj_val.item()))
        times_ADMM.append(float(time.time() - start_time))
        value_ADMM.append(obj_val.item())
    t+=1
print()

iter= 0,	objective= 647100.845969


KeyboardInterrupt: ignored